<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/pellet_count_wrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# Source

## 2011 - 2018
* dataset: ‘[to fall 2018 Biomass4](https://docs.google.com/spreadsheets/d/1j-5Wfuueyr8Zh39MXnDSZuPbwEHnriICf7sIx4ZQosw/edit?usp=sharing)’
  * sheet: ‘[biomass per m^2 spss](https://docs.google.com/spreadsheets/d/1j-5Wfuueyr8Zh39MXnDSZuPbwEHnriICf7sIx4ZQosw/edit?usp=sharing)’


In [0]:
# to fall 2018 Biomass4 - biomass per m^2 spss.csv
# src = 'https://drive.google.com/uc?id=1KJRC9tTm5f-NJvIkNkfDIJ4YKPx4Prkk'

[Read Google Spreadsheet data into Pandas Dataframe](https://kanoki.org/2018/12/25/read-google-spreadsheet-data-into-pandas-dataframe/)

In [0]:
# Google Sheet: 'to fall 2018 Biomass4 - biomass per m^2 spss'
# url = 'https://docs.google.com/spreadsheets/d/1j-5Wfuueyr8Zh39MXnDSZuPbwEHnriICf7sIx4ZQosw/edit#gid=1101523461'
src = 'https://docs.google.com/spreadsheets/d/1j-5Wfuueyr8Zh39MXnDSZuPbwEHnriICf7sIx4ZQosw/export?format=csv&gid=1101523461'

In [0]:
df = pd.read_csv(src)

In [0]:
df.head()

,GridPt,Year,Period,bmsmerm2,group_num,group,PropGrass,Elk_Pellets,Deer_Pellets,Horse_Pellets,Robel,%Forbs,pounds per acre,rain,prev_rain,GridPt.1,Year.1,Period.1
0,3,2010,2,28.8,1,Range,NaN,NaN,NaN,NaN,NaN,NaN,285.7,16,11,3,2010,2
1,4,2010,2,8.1,1,Range,NaN,NaN,NaN,NaN,NaN,NaN,80.0,16,11,4,2010,2
2,5,2010,2,31.2,1,Range,NaN,NaN,NaN,NaN,NaN,NaN,309.8,16,11,5,2010,2
3,6,2010,2,9.6,1,Range,NaN,NaN,NaN,NaN,NaN,NaN,95.6,16,11,6,2010,2
4,7,2010,2,10.5,1,Range,NaN,NaN,NaN,NaN,NaN,NaN,104.5,16,11,7,2010,2


In [0]:
df = df[['GridPt', 'Year', 'Period', 'Elk_Pellets', 'Deer_Pellets', 'Horse_Pellets']]

In [0]:
df.head()

,GridPt,Year,Period,Elk_Pellets,Deer_Pellets,Horse_Pellets
0,3,2010,2,NaN,NaN,NaN
1,4,2010,2,NaN,NaN,NaN
2,5,2010,2,NaN,NaN,NaN
3,6,2010,2,NaN,NaN,NaN
4,7,2010,2,NaN,NaN,NaN


## 2019
* dataset: '[2019 Biomass Data.xlsx](https://drive.google.com/file/d/1dd5F8R-mD6l7H2morac1r9ongf80xpff/view?usp=sharing)'
  * sheet: 'All Data - redone'

In [0]:
# 2019 Biomass Data - All Data - redone.csv
src_19 = 'https://drive.google.com/uc?id=1dAv09PPnGgd4KlQ5YTQ1gKPjIgt3-cts'

In [0]:
df_19 = pd.read_csv(src_19)

In [0]:
# select columns of interest
df_19 = df_19[['Plot', 'Date', 'Deer', 'Elk', 'Horse']]

In [0]:
# rename columns
df_19 = df_19.rename(columns={'Plot':'grid_point', 'Date':'date', 'Deer':'deer', 'Elk':'elk', 'Horse':'horse'})

In [0]:
# cast date to 'datetime' datatype
df_19['date'] = pd.to_datetime(df_19.date)

In [0]:
df_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 5 columns):
grid_point    347 non-null int64
date          325 non-null datetime64[ns]
deer          319 non-null float64
elk           319 non-null float64
horse         319 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 13.7 KB


In [0]:
df_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 5 columns):
grid_point    347 non-null int64
date          325 non-null datetime64[ns]
deer          319 non-null float64
elk           319 non-null float64
horse         319 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 13.7 KB


In [0]:
for index, row in df_19.iterrows():
  if row['date'] < pd.Timestamp('2019-06-01 00:00:00'):
    df_19.loc[index, 'season'] = 'spring'
  elif row['date'] > pd.Timestamp('2019-06-01 00:00:00'):
    df_19.loc[index, 'season'] = 'fall'

In [0]:
# convert DataFrame from wide to long format
df_19 = pd.melt(df_19, id_vars=['grid_point', 'date', 'season'], value_vars=['elk','deer','horse'],
        var_name='species', value_name='pellet_clusters_count')

### Remove/revise NA

In [0]:
# remove rows with null values
df_19 = df_19[(df_19.pellet_clusters_count.notnull())]

In [0]:
df_19[df_19.date.isnull()]

,grid_point,date,season,species,pellet_clusters_count
102,197,NaT,NaN,elk,0.0
449,197,NaT,NaN,deer,1.0
796,197,NaT,NaN,horse,1.0


In [0]:
# update row with 'NaT' 'NaN' values
df_19.loc[df_19.date.isnull(), 'season'] = 'fall'
df_19.loc[df_19.date.isnull(), 'date'] = '2019-09-04'

In [0]:
# check the result
df_19[df_19.grid_point == 197]

,grid_point,date,season,species,pellet_clusters_count
102,197,2019-09-04,fall,elk,0.0
449,197,2019-09-04,fall,deer,1.0
796,197,2019-09-04,fall,horse,1.0


### Add year column

In [0]:
df_19.loc[:, 'year'] = 2019

In [0]:
# reorder columns and drop date
df_19 = df_19[['grid_point', 'year', 'season', 'species', 'pellet_clusters_count']]

## 2020

In [0]:
# 2020-04-21_spring_biomass_data - Sheet1.csv
src_2020 = 'https://drive.google.com/uc?id=1Q1SvzROHySWxIjG7qSFYK9luwJB6RIJ0'

# Wrangle

## 2011 - 2018

In [0]:
# remove 2010
df = df[(df['Year'] >= 2011)]

In [0]:
# rename period to season
df = df.rename(columns={"GridPt":"grid_point", "Year":"year", "Elk_Pellets":"elk", 
                        "Deer_Pellets":"deer", "Horse_Pellets":"horse"})

In [0]:
df = pd.melt(df, id_vars=['grid_point', 'year', 'Period'], value_vars=['elk', 'deer', 'horse'], 
             var_name='species', value_name='pellet_clusters_count')

In [0]:
# classify 'season' based on 'Period'
for index, row in df.iterrows():
  if row['Period'] == 1:
    df.loc[index, 'season'] = 'spring'
  if row['Period'] == 2:
    df.loc[index, 'season'] = 'fall'

In [0]:
# drop 'Period' column
df = df[['grid_point', 'year', 'season', 'species', 'pellet_clusters_count']]

In [0]:
df[df['year'] == 2011].head()

,grid_point,year,season,species,pellet_clusters_count
0,2,2011,spring,elk,NaN
1,3,2011,spring,elk,NaN
2,4,2011,spring,elk,NaN
3,5,2011,spring,elk,NaN
4,6,2011,spring,elk,NaN


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5115 entries, 0 to 5114
Data columns (total 5 columns):
grid_point               5115 non-null int64
year                     5115 non-null int64
season                   5115 non-null object
species                  5115 non-null object
pellet_clusters_count    4202 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 199.9+ KB


### Remove/revise NA


In [0]:
df = df[(df.pellet_clusters_count.notnull())]

## 2019

## Combine 2011 - 2018 and 2019

In [0]:
df.head()

,grid_point,year,season,species,pellet_clusters_count
69,2,2011,fall,elk,1.0
70,3,2011,fall,elk,2.0
71,4,2011,fall,elk,3.0
72,5,2011,fall,elk,2.0
73,6,2011,fall,elk,0.0


In [0]:
df_19.head()

,grid_point,year,season,species,pellet_clusters_count
0,9,2019,fall,elk,3.0
1,10,2019,fall,elk,1.0
2,11,2019,fall,elk,1.0
3,12,2019,fall,elk,2.0
4,19,2019,fall,elk,0.0


In [0]:
frames = [df, df_19]
result = pd.concat(frames)

In [0]:
result.tail()

,grid_point,year,season,species,pellet_clusters_count
1036,559,2019,spring,horse,0.0
1037,560,2019,spring,horse,0.0
1038,561,2019,spring,horse,0.0
1039,571,2019,spring,horse,0.0
1040,572,2019,spring,horse,0.0


## Wide format

In [0]:
wide = result.set_index(['grid_point', 'year', 'season', 'species']).unstack(level=-1).reset_index()

In [0]:
wide.columns = ['grid_point', 'year', 'season', 'deer', 'elk', 'horse']

In [0]:
wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 6 columns):
grid_point    1946 non-null int64
year          1946 non-null int64
season        1946 non-null object
deer          1946 non-null float64
elk           1946 non-null float64
horse         1267 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 91.3+ KB


In [0]:
wide.tail()

,grid_point,year,season,deer,elk,horse
1941,571,2019,spring,0.0,0.0,0.0
1942,572,2019,fall,0.0,0.0,0.0
1943,572,2019,spring,0.0,0.0,0.0
1944,574,2015,fall,2.0,4.0,0.0
1945,574,2015,spring,5.0,0.0,0.0


In [3]:
# output 2020-03-24
# used as source for BigQuery table 'pellet_count'
wide.to_csv("pellet_count-2011_2019.csv", index=None

SyntaxError: ignored

## 2020


In [0]:
# load 2020 data into DataFrame
df_2020 = pd.read_csv(src_2020)

In [0]:
# select columns pertinent to pellet counts
df_2020 = df_2020.loc[:, ['Plot', 'year', 'season', 'pellets_Deer', 'pellets_Elk', 'pellets_Horse']]

In [0]:
# rename columns
df_2020.columns = ['grid_point', 'year', 'season', 'deer', 'elk', 'horse']

In [0]:
# update columns datatypes to match BigQuery Table
# 'season' to string
df_2020['season'] = df_2020['season'].astype(str)

# ['deer', 'elk', 'horse'] to float
df_2020["deer"] = pd.to_numeric(df_2020["deer"], downcast="float")
df_2020["elk"] = pd.to_numeric(df_2020["elk"], downcast="float")
df_2020["horse"] = pd.to_numeric(df_2020["horse"], downcast="float")

# Update

BigQuery Table: [mpg-data-warehouse:vegetation_biomass.pellet_count](https://console.cloud.google.com/bigquery?project=mpg-data-warehouse&p=mpg-data-warehouse&d=vegetation_biomass&t=pellet_count&page=table)

## 2020 Spring

In [45]:
df_2020.head(2)

,grid_point,year,season,deer,elk,horse
0,9,2020,spring,0.0,2.0,0.0
1,10,2020,spring,2.0,3.0,1.0


In [46]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   grid_point  162 non-null    int64  
 1   year        162 non-null    int64  
 2   season      162 non-null    object 
 3   deer        162 non-null    float32
 4   elk         162 non-null    float32
 5   horse       162 non-null    float32
dtypes: float32(3), int64(2), object(1)
memory usage: 5.8+ KB


In [0]:
# append 2020 Spring DataFrame to BigQuery table
df_2020.to_gbq('vegetation_biomass.pellet_count', 'mpg-data-warehouse', if_exists='append')